In [2]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from google.cloud import bigquery
import pandas as pd
import time
import zipfile
import datetime
from datetime import date
from tqdm import tqdm

# Initiate BigQuery client
client = bigquery.Client()

Using Kaggle's public dataset BigQuery integration.


In [3]:
# Store the contract addresses
address_dict = dict()

address_dict['LUSD'] = '0x5f98805A4E8be255a32880FDeC7F6728C6568bA0'
address_dict['LQTY'] = '0x6DEA81C8171D0bA574754EF6F8b412F2Ed88c54D'
address_dict['AAVE'] = '0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9'
address_dict['COMP'] = '0xc00e94Cb662C3520282E6f5717214004A7f26888'
address_dict['Dai']  = '0x6B175474E89094C44Da98b954EedeAC495271d0F'

# Lower case the contract addresses
for k in address_dict:
    address_dict[k] = address_dict[k].lower()

In [4]:
def get_tx_df(
        token_name, 
        last_date = (date.today()- datetime.timedelta(days=1)).strftime("%Y-%m-%d")
    ):
    # Create a "Client" object
    client = bigquery.Client()

    # Construct a reference to the "crypto_ethereum" dataset (https://www.kaggle.com/bigquery/ethereum-blockchain)
    dataset_ref = client.dataset("crypto_ethereum", project="bigquery-public-data")

    # API request - fetch the dataset
    dataset = client.get_dataset(dataset_ref)

    # List all the tables in the "crypto_ethereum" dataset
    tables = list(client.list_tables(dataset))

    sql = '''
        SELECT 
          token_address, from_address, to_address, 
          block_timestamp AS timestamp, CAST(value AS NUMERIC) AS value
        FROM `bigquery-public-data.crypto_ethereum.token_transfers` 
        WHERE token_address = '{}'
          AND block_timestamp < cast('{} 00:00:00' as TIMESTAMP)
    '''.format(address_dict[token_name], last_date)

    df = client.query(sql).to_dataframe(progress_bar_type='tqdm_notebook')

    # Clean
    df['timestamp'] = pd.to_datetime(df['timestamp']).dt.date
    df.sort_values(by='timestamp', ascending=False, inplace=True)

    # Save
    file_name = f"{token_name}_{last_date}.csv"
    df.to_csv(file_name, index=False)
    print(f'Saved to {file_name}')
    return df

In [55]:
# Query the transaction records for one specific protocal and an end date
get_tx_df('LUSD', '2022-07-13')

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Downloading:   0%|          | 0/211809 [00:00<?, ?rows/s]

Saved to LUSD_2022-07-13.csv


,token_address,from_address,to_address,timestamp,value
136104,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,0xed279fdd11ca84beef15af5d39bb4d4bee23f0ca,0x22f9dcf4647084d6c31b2765f6910cd85c178c18,2022-07-12,503678619561092193149531.000000000
88802,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,0x16980c16811bde2b3358c1ce4341541a4c772ec9,0x4cbc25223ae8076f6b509593a383d395e1adf3a5,2022-07-12,9466206988831983756895.000000000
90996,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,0x4f9fbb3f1e99b56e0fe2892e623ed36a76fc605d,0xf13a45ff5763cb77fe3dcaaaa25652d15f95aef2,2022-07-12,0E-9
90997,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,0x4f9fbb3f1e99b56e0fe2892e623ed36a76fc605d,0x47f80003f5dfa14c12d25dd52c2bbab9f206504e,2022-07-12,0E-9
90998,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,0x66017d22b0f8556afdd19fc67041899eb65a21bb,0x00ff66ab8699aafa050ee5ef5041d1503aa0849a,2022-07-12,182818598842874926079793.000000000
...,...,...,...,...,...
164021,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,0x9d616e9d0b074a4aa1106c68f13f54054ed22382,0x66017d22b0f8556afdd19fc67041899eb65a21bb,2021-04-05,4000000000000000000000.000000000
164022,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,0xbb4946eea34a98b2c0e497cb7f8f3af83311b2af,0x66017d22b0f8556afdd19fc67041899eb65a21bb,2021-04-05,34626865671641791044777.000000000
164023,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,0x1c802b1f27acd3a3def732899fd1f4c61fe30dd8,0x66017d22b0f8556afdd19fc67041899eb65a21bb,2021-04-05,13619900497512437810946.000000000
164024,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,0x95c231ff84476aa15ce7e57aee153ba2446f7e82,0x66017d22b0f8556afdd19fc67041899eb65a21bb,2021-04-05,298308457711442786069652.000000000


In [5]:
# Query the transaction records for all 6 DeFi tokens
for key, _ in address_dict.items():
    print(key)
    get_tx_df(key, '2022-07-13')

LUSD
Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Downloading:   0%|          | 0/211809 [00:00<?, ?rows/s]

Saved to LUSD_2022-07-13.csv
LQTY
Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


...

Downloading:   0%|          | 0/1799967 [00:00<?, ?rows/s]

Saved to COMP_2022-07-13.csv
Dai
Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Downloading:   0%|          | 0/14587616 [00:00<?, ?rows/s]

Saved to Dai_2022-07-13.csv
